In [5]:
import gurobipy as gp
from gurobipy import GRB


# prompt: create model and decision variable with size 40 x 1

m = gp.Model("5-2-1")

# Create decision variables


# Create decision variables using a list of indices from 1 to 40
indices = list(range(0, 40))
indices_2 = list(range(0, 4))

x = m.addVars(indices, vtype=GRB.BINARY, name='x')
c = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='c')
z = m.addVars(indices, vtype=GRB.INTEGER, name='z')
y = m.addVars(indices, lb = 0, vtype=GRB.CONTINUOUS, name='y')
beta = m.addVars(indices_2, lb = 0, vtype=GRB.INTEGER, name='Beta')  # Beta_1, Beta_2, Beta_3, Beta_4


# Objective function
m.setObjective(y.sum())



# Define the alpha matrix
import itertools

# Card values from 1 to 5
card_values = range(1, 6)

# Generate all combinations for face-up card with two positions (0 = left, 1 = right)
face_up_combinations = [(value, pos) for value in card_values for pos in [0, 1]]

# Generate alpha matrix
alpha_matrix = []

for (face_up_value, face_up_pos) in face_up_combinations:
    for hidden_value in card_values:
        if hidden_value != face_up_value:  # The hidden card must be different
            alpha_i1 = face_up_value
            alpha_i2 = face_up_pos
            alpha_j3 = hidden_value
            alpha_matrix.append([alpha_i1, alpha_i2, alpha_j3])

# Print 4 entries per row
for index in range(0, len(alpha_matrix), 4):
    print(alpha_matrix[index:index+4])

# Create constraints

P = 5

for i in range(40):
    m.addConstr(y[i] + (1 - x[i]) * P >= c[i])

for i in range(10):
  m.addConstr(gp.quicksum(x[4*i + j] for j in range(4)) <= 1, 'x_sum_{}'.format(i))

for i in range(40):
    m.addConstr(c[i] == beta[0] + gp.quicksum(beta[k+1] * alpha_matrix[i][k] for k in range(0, 3)) - z[i] * P)


pair_sums = {}

for i, (val1, _, val3) in enumerate(alpha_matrix):
    pair_key = frozenset([val1, val3])
    if pair_key not in pair_sums:
        pair_sums[pair_key] = x[i]
    else:
        pair_sums[pair_key] += x[i]

# Now, add a constraint for each sum in pair_sums to be equal to 1
for pair_key, x_sum in pair_sums.items():
    sorted_key = sorted(list(pair_key))
    m.addConstr(x_sum == 1, f"pair_sum_equals_1_{'_'.join(map(str, sorted_key))}")


# Additional Cuts
m.addConstr(beta.sum() >= 1)
for i in range(4):
  m.addConstr(beta[i] <= P-1)


m.optimize()


# Print the coeeficients
for i in range(4):
    print('beta[{}]: {}'.format(i, beta[i].X))


# Initialize a counter for zero_cost links
zero_count = 0

for i in range(40):
     if c[i].X == 0.0:
        zero_count += 1

print(zero_count)


# Print the values of the 'c' variables
for i in range(40):
    print('x[{}]: {}'.format(i, x[i].X), 'c[{}]: {}'.format(i, c[i].X))


[[1, 0, 2], [1, 0, 3], [1, 0, 4], [1, 0, 5]]
[[1, 1, 2], [1, 1, 3], [1, 1, 4], [1, 1, 5]]
[[2, 0, 1], [2, 0, 3], [2, 0, 4], [2, 0, 5]]
[[2, 1, 1], [2, 1, 3], [2, 1, 4], [2, 1, 5]]
[[3, 0, 1], [3, 0, 2], [3, 0, 4], [3, 0, 5]]
[[3, 1, 1], [3, 1, 2], [3, 1, 4], [3, 1, 5]]
[[4, 0, 1], [4, 0, 2], [4, 0, 3], [4, 0, 5]]
[[4, 1, 1], [4, 1, 2], [4, 1, 3], [4, 1, 5]]
[[5, 0, 1], [5, 0, 2], [5, 0, 3], [5, 0, 4]]
[[5, 1, 1], [5, 1, 2], [5, 1, 3], [5, 1, 4]]
Gurobi Optimizer version 11.0.1 build v11.0.1rc0 (win64 - Windows 11.0 (22621.2))

CPU model: Intel(R) Core(TM) i7-1065G7 CPU @ 1.30GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 105 rows, 164 columns and 428 nonzeros
Model fingerprint: 0x99c623de
Variable types: 80 continuous, 84 integer (40 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00,